In [3]:
import json
import random
import os

# Define the control point keys
keys = [
    "originalSvCenter", "svCenter", "linX", "linY", "linZ", "edgeXY", "edgeYZ", "edgeXZ",
    "mob", "int1", "int2", "int3", "int4", "int5", "int6",
    "int7", "int8", "int9", "int10", "int11", "int12"
]

# Base center for originalSvCenter
base_center = [30.0, 30.0, 30.0]

def random_point_around(base, spread=5.0):
    return [base[i] + random.uniform(-spread, spread) for i in range(3)]

data = []
for _ in range(200):
    control_points = {}
    control_points["originalSvCenter"] = base_center[:]
    for k in keys:
        if k == "originalSvCenter":
            continue
        # Each point is random around the base center
        control_points[k] = random_point_around(base_center, spread=5.0)
    data.append({"control_points": control_points})

# Ensure the output directory exists
os.makedirs("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/debug_viz", exist_ok=True)

# Write to JSON file
with open("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/debug_viz/sv333_debug_synth.json", "w") as f:
    json.dump(data, f, indent=2)

In [3]:
MASTER_ALGORITHM_PROMPT=""" You are an expert in computational geometry and procedural mesh generation, acting as an algorithmic design assistant. Your task is to generate a sequence of algorithmic steps, conforming strictly to the provided EBNF grammar, to define control points and triangles for a single supervoxel  in a 3D grid. The goal is to create a watertight, star-domain polyhedron for supervoxel, formed by tetrahedrons sharing sv_center as their common apex. Also algorithm need to be divided into steps and those steps surrounded by tags !.
1. Grammar Compatibility Analysis
To be valid, the generated algorithm must be parsable by the provided EBNF grammar. Below are the key implications of EBNF Grammar grammar.

```
?start: statement_list
?statement_list: (statement)*
?statement: assign_var ";"
          | triangle_def ";"
          | expr ";" -> expr_statement
?assign_var: NAME "=" expr
?triangle_def: "defineTriangle" "(" arguments ")" -> triangle_definition
?expr: sum
?sum: product
    | sum "+" product -> add
    | sum "-" product -> sub
?product: power
        | product "*" power -> mul
        | product "/" power -> div
?power: atom_expr "^" power -> pow
      | atom_expr
?atom_expr: func_call
          | atom
          | "-" atom_expr -> neg
          | "(" expr ")"
?atom: NUMBER -> number
     | indexed_var
     | NAME -> var
     | point
point: "(" expr "," expr "," expr ")"
?indexed_var: NAME "_" "(" arguments ")"
func_call: FUNC_NAME "(" [arguments] ")"
FUNC_NAME.2: "sin" | "cos" | "sqrt" | "dot" | "div" | "vector" | "lin_p" | "ortho_proj" | "line_plane_intersect" | "range"
arguments: expr ("," expr)*
NAME: /(?!(sin|cos|sqrt|dot|div|vector|lin_p|ortho_proj|line_plane_intersect|range)$)[a-zA-Z][a-zA-Z0-9]*/
%import common.NUMBER
%import common.WS
%ignore WS
%ignore /#.*/
```

A. Core Syntax Rules
Statements: Every line of code that performs an action (assignment, function call) is a statement and must end with a semicolon (;).
Variable Naming (NAME): All user-defined variables (for points or scalars) must be camelCase. They must start with a letter and contain only letters and numbers (e.g., svCenter, temp1, myPoint). Underscores are not permitted in variable names.
Function Naming (fun_name_rule): Pre-defined function names are specific lowercase tokens and must be used exactly as listed (e.g., lin_p, ortho_proj, vector). defineTriangle is an exception and is camelCase.
Indexing (indexed_var): To access any control point, you must use an underscore (_) followed by parentheses containing three comma-separated integer offsets (dx, dy, dz). This is mandatory for all point references.
To access a point from the current supervoxel, use the index _(0,0,0) (e.g., mob_(0,0,0)).
To access a point from a neighboring supervoxel, use its relative index (e.g., mob_(1,0,0)).
Dependency Rule: When defining a control point in a given step (e.g., mob in Step 3), you can only access control points from neighboring supervoxels that were defined in previous steps. You cannot access a point from a neighbor that is defined in the same step. For example, the definition of mob cannot reference mob_(-1,0,0), as all mob points are considered to be calculated simultaneously within Step 3. It can, however, reference linX_(-1,0,0) (from Step 2) or svCenter_(-1,0,0) (from Step 1).
Weight Usage: Weights (w1, w2, w3, etc.) must be used in sequential order, starting from w1. Do not skip weights (e.g., do not use w3 without first using w1 and w2).
Comments: Comments are ignored by the parser and can be added using #.
B. Allowed Operations and Functions
The grammar only permits the following operations. No other functions or operators are available.
Arithmetic: + (addition), - (subtraction), * (multiplication), / (division), ^ (power).
Vector/Point Constructors:
vector(x, y, z); # Creates a 3D vector or point.
Linear Interpolation:
lin_p(pointA, pointB, weight); # Returns a point interpolated between pointA and pointB.
Geometric Projections:
ortho_proj(pointToProject, planeP1, planeP2, planeP3); # Projects a point onto a plane.
line_plane_intersect(lineP1, lineP2, planeP1, planeP2, planeP3); # Calculates the intersection of a line (defined by two points, lineP1 and lineP2) and a plane (defined by three points, planeP1, planeP2, planeP3). All arguments are points (tuples of 3 floats).
Mathematical Functions:
sin(expr);, cos(expr);, sqrt(expr);
Vector Operations:
dot(vectorA, vectorB); # Returns the scalar dot product.
div(vectorA, scalarB); # Performs scalar division on a vector.
Triangle Definition:
defineTriangle(p1, p2, p3); # Defines a triangle face for the final mesh.
C. Prohibited Constructs
The following are NOT compatible with the grammar and must not be used:
Conditional Logic: if, else, switch, ternary operators (? :).
Boolean Logic: ==, !=, <, >, &&, ||.
Comparison Functions: min(), max(), abs().
Loops: for, while, do-while.
Data Structures: Arrays, lists, or explicit object structures (e.g., [ ] or { }).
2. Core Design Pattern: The "Negative Corner"
To ensure a watertight mesh across all supervoxels, your algorithm must follow a "Negative Corner" design pattern. This is a fundamental concept for this task.
Control Point Definition: You will only define the control points for one specific corner out of the eight corners of the supervoxel cube: the one located in the (-x, -y, -z) direction relative to its originalSvCenter. All points you define in steps <S2>, <S3>, and <S4> (e.g., linX, edgeXY, mob) belong exclusively to this single "Negative Corner".
Triangle Assembly: In step <S5>, you will construct the full, six-sided polyhedron. This is done by defining triangles for both the negative faces (using the points you defined for the current supervoxel) and the positive faces (by accessing the "Negative Corner" points from neighbors in the positive directions, like linX_(1,0,0)).
Dependency Rule: When defining a control point, you can only access control points from neighboring supervoxels if those variables were defined in a previous step. You can not access any temporary variables from previous steps.The set of accessible neighbor variables grows with each step:
Always Available: You can always access a neighbor's originalSvCenter (e.g., originalSvCenter_(-1,0,0)).
During Step 2: You can access originalSvCenter and svCenter from neighbors.
During Step 3: You can access originalSvCenter, svCenter, and all points from Step 2 (linX, linY, linZ, edgeXY, etc.) from neighbors. You cannot access a neighbor's mob point.
During Step 4: You can access all points from Steps 1, 2, and 3 from neighbors (e.g., svCenter, linX, mob). You cannot access a neighbor's int points.
During Step 5: All control points (svCenter, lin*, edge*, mob, int*) are considered fully defined and can be accessed from any necessary neighbor.
This pattern guarantees that any shared face between two supervoxels is defined by the exact same set of control points, leading to a perfectly sealed and valid mesh.
3. Post-Generation Algorithm Evaluation
Once the algorithm is generated, it will undergo rigorous automated testing using multiple sets of random weights to validate its robustness and correctness. An algorithm fails if it does not pass all three of the following tests:
Watertight Mesh Verification: The surface mesh for every supervoxel must be perfectly sealed (watertight). The test will check for any gaps or holes in the mesh. A non-watertight surface will result in failure.
Star Domain and Self-Intersection Test: The volume defined by the supervoxel's tetrahedrons must maintain the star domain property with respect to svCenter. A simplified test will be performed: for each triangle, a ray will be cast from svCenter through the triangle's barycenter. This ray must not intersect any other triangle in the same supervoxel. Any self-intersection indicates a failed algorithm.
Flexibility and Degeneracy Analysis: An LLM will qualitatively assess the algorithm's flexibility. The primary goal is to create shapes with maximum adaptability. The analysis will check for geometric degeneracies, such as unnecessary co-linearity between control points (beyond the intended linX, linY, linZ points), which would improperly constrain the shape and limit its expressive power.
4. Algorithm Generation Prompt
You are to generate the body of the algorithm. The strict grammar rules detailed in Section 1 and the Negative Corner pattern from Section 2 apply only to the code generated within the <S1> through <S5> blocks. Text outside these blocks is for instruction only. Comments (lines starting with #) are allowed and encouraged within the code blocks as they are ignored by the parser.
The process is divided into distinct, tagged steps. For each step, you must provide the complete code block, ensuring every line terminates with a semicolon.
Step 0: Preparation (Context, Not for Generation)
Before your task begins, assume the following are pre-defined and available for use:
Grid Spacing: rx, ry, rz (scalar values for grid dimensions).
Weights: w1 through w150 (scalar values between 0 and 1).
Original Centers: originalSvCenter (the center point of the current supervoxel) and its neighbors accessible via relative indexing (e.g., originalSvCenter_(-1, 0, 0)).
Step 1: Supervoxel Center Modification (Tag: <S1>)
Task: Define the modified center point, svCenter. This point will serve as the common apex for all tetrahedrons in the supervoxel.
Constraint: svCenter must be displaced from originalSvCenter by a maximum of (rx*0.5, ry*0.5, rz*0.5). Use weights to control this displacement.
<S1>
# The following is just one possible implementation example.
svCenter = originalSvCenter_(0,0,0) + vector(w1 * rx * 0.5 - rx * 0.25, w2 * ry * 0.5 - ry * 0.25, w3 * rz * 0.5 - rz * 0.25);
</S1>

Step 2: Linear and Edge Control Points (Tag: <S2>)
Task: Define the primary control points for the "Negative Corner" of the supervoxel.
linX, linY, linZ: Points on the segments connecting the current originalSvCenter to its negative neighbors (-1,0,0), (0,-1,0), and (0,0,-1). Use lin_p and weights.
edgeXY, edgeYZ, edgeXZ: Points that define the corners of the negative octant. These should be derived from the originalSvCenter and its neighbors (e.g., edgeXY from (0,0,0), (-1,0,0), (0,-1,0), (-1,-1,0)).
<S2>
# Your implementation here...
</S2>

Step 3: Main Oblique Point Definition (Tag: <S3>)
Task: Define the mob (main oblique point) control point, which is the innermost point of the "Negative Corner".
Derivation: mob must be a complex blend of the originalSvCenter from the current supervoxel and its seven neighbors in the negative directions (e.g., _(-1,0,0), _(-1,-1,0), _(-1,-1,-1), etc.).
Purpose: This point provides high-level control over the interior corner of the supervoxel. Use a combination of the available functions with different weights to achieve a flexible blend.
Dependency Note: The definition for mob can only use points defined in Steps 1 and 2 from its neighbors (e.g., linX_(-1,0,0)), not the mob point from a neighbor.
<S3>
# Your implementation here...
</S3>

Step 4: Intermediate Points Definition (Tag: <S4>)
Task: Define all remaining control points (int1 through int12) for the "Negative Corner".
Derivation: Use the full range of available functions (ortho_proj, line_plane_intersect, dot, etc.) and previously defined points to create these. Their purpose is to add geometric complexity and adaptability.
Dependency Note: Remember you can only reference points from neighbors that were defined in steps 1, 2, and 3.
<S4>
# Your implementation here...
</S4>

Step 5: Triangle Definitions (Tag: <S5>)
Task: Define all triangles necessary to form a complete and watertight mesh for the supervoxel by assembling its six faces . Make sure that all points ["linX", "linY", "linZ", "mob", "int1", "int2", "int3", "int4", "int5", "int6", "int7", "int8", "int9", "int10", "int11", "int12"] are used .
Assembly Strategy: Following the "Negative Corner" pattern, you will construct the full mesh by defining the triangles for each of its six faces. For each face, you must use the appropriate set of control points with their explicit indices.
1. Negative-X Face: Define the triangles for the face on the -x side.
Points: This face is part of the current supervoxel's "Negative Corner". Its triangles are primarily defined by points from the current supervoxel (e.g., linY_(0,0,0), linZ_(0,0,0), mob_(0,0,0), edgeYZ_(0,0,0)) and points shared with neighbors in the negative-Y and negative-Z directions (e.g., linY_(0,-1,0)).
2. Negative-Y Face: Define the triangles for the -y face.
Points: This face is also part of the "Negative Corner". It uses points from the current supervoxel (e.g., linX_(0,0,0), linZ_(0,0,0)) and those shared with neighbors in the negative-X and negative-Z directions.
3. Negative-Z Face: Define the triangles for the -z face.
Points: This face is the final part of the "Negative Corner". It uses points from the current supervoxel (e.g., linX_(0,0,0), linY_(0,0,0)) and those shared with neighbors in the negative-X and negative-Y directions.
4. Positive-X Face: Define the triangles for the +x face.
Points: This face is the Negative-X face of the neighbor at (1,0,0). You must build its triangles by accessing the complete "Negative Corner" point set from that neighbor.
Example: defineTriangle(linY_(1,0,0), linZ_(1,0,0), mob_(1,0,0));
5. Positive-Y Face: Define the triangles for the +y face.
Points: This is the Negative-Y face of the neighbor at (0,1,0). Access that neighbor's "Negative Corner" points to build the triangles.
6. Positive-Z Face: Define the triangles for the +z face.
Points: This is the Negative-Z face of the neighbor at (0,0,1). Access that neighbor's "Negative Corner" points to build the triangles.
Watertight Rule: This face-by-face assembly, based on the Negative Corner pattern, inherently enforces the watertight rule.
Star Domain: All defined triangles form the base of tetrahedrons whose apex is svCenter.
IMPORTANT on each face there are always triangles that have points from different neighbours so it is NEVER the case that all of the points on a face a re from single supervoxel.
<S5>
# Your implementation here...
</S5>


MOST IMPORTANT remember to wrap steps in tags <S1></S1> ; <S2></S2> ; <S3></S3>; <S4></S4> ;<S5></S5> !! ."""

import json

log_path = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/vllm_api_log.json"
with open(log_path, "r") as f:
    vllm_data = [json.loads(line) for line in f if line.strip()]

# Check if all objects have the same set of keys
all_keys = [set(obj.keys()) for obj in vllm_data]
same_keys = all(k == all_keys[0] for k in all_keys)

print("All objects have the same keys:", same_keys)
if not same_keys:
    for i, keys in enumerate(all_keys):
        print(f"Object {i} keys: {keys}")
else:
    print("Keys:", all_keys[0])


All objects have the same keys: True
Keys: {'full_prompt_sent', 'context', 'prompt', 'call_name'}


In [10]:
import json
import os
import math

def create_markdown_from_log(log_path="vllm_log.jsonl", output_dir="output_markdown"):
    """
    Reads a log file with JSON objects, divides them into 10 parts,
    and creates a separate Markdown file for each part in a specified directory.

    This function assumes the log file exists and contains valid JSON.

    Args:
        log_path (str): The path to the input log file.
        output_dir (str): The path to the directory where output files will be saved.
    """
    # Step 1: Read the log file and parse all JSON data
    # This will raise an error if the file is not found or is invalid.
    with open(log_path, "r") as f:
        vllm_data = [json.loads(line) for line in f if line.strip()]

    # Step 2: Create the output directory if it doesn't already exist
    # The `exist_ok=True` argument prevents an error if the directory is already there.
    os.makedirs(output_dir, exist_ok=True)

    # Step 3: Calculate the size of each of the 10 chunks
    total_objects = len(vllm_data)
    if total_objects == 0:
        print("Log file is empty. No output files will be created.")
        return

    # Use math.ceil to ensure all objects are included, even if not perfectly divisible by 10.
    # For example, 95 objects would result in 10 chunks of size 10.
    chunk_size = math.ceil(total_objects / 10)
    if chunk_size == 0:
        print("No objects to process.")
        return

    # Step 4: Iterate 10 times to create 10 files
    files_created_count = 0
    for i in range(10):
        # Define the start and end index for the current chunk of data
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        
        # Slice the main list to get the current chunk
        data_chunk = vllm_data[start_index:end_index]

        # If the start index is beyond the list length, this chunk will be empty, so we can stop.
        if not data_chunk:
            break

        markdown_sections = []
        # Process each object within the current chunk
        for j, obj in enumerate(data_chunk):
            if "call_name" not in obj or "full_prompt_sent" not in obj:
                original_index = start_index + j
                print(f"Warning: Skipping object at original index {original_index} because it lacks 'call_name' or 'full_prompt_sent'.")
                continue

            title = obj.get("call_name", "Untitled Section")
            content = obj.get("full_prompt_sent", "")

            # Remove the specified string from the content
            cleaned_content = content.replace(MASTER_ALGORITHM_PROMPT, "").strip()

            # Format the entry as a markdown section
            section = f"# {title}\n\n{cleaned_content}\n\n---\n"
            markdown_sections.append(section)
        
        # If there's content to write for this chunk, save it to a file
        if markdown_sections:
            final_markdown = "".join(markdown_sections)
            
            # Construct the full path for the output file (e.g., "output_markdown/part_1.md")
            output_path = os.path.join(output_dir, f"part_{i + 1}.md")

            with open(output_path, "w") as f:
                f.write(final_markdown)
            
            files_created_count += 1

    print(f"Successfully created {files_created_count} Markdown files in the directory: {output_dir}")
    print(f"Processed {total_objects} total objects.")



log_path = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/vllm_api_log.json"
output_path = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/output_markdown"

create_markdown_from_log(log_path, output_path)


Successfully created 10 Markdown files in the directory: /workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/output_markdown
Processed 522 total objects.


In [ ]:
# import json
# log_path = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/vllm_api_log.json"
# output_path = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/vllm_api_log_no_prompt.json"
# with open(log_path, "r") as f:
#     vllm_data = [json.loads(line) for line in f if line.strip()]


# print(f"Cleaned log saved to: {output_path}")

Cleaned log saved to: /workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/constrained_vllm_python/main_debug/vllm_api_log_no_prompt.json
